# Welcome to *LatEpig* v2.0

*The Jupyter Notebook inteface for the **LatEpig** tool allows you to query all the inscriptions from the Epigraphic Database Clauss Slaby (www.manfredclauss.de) in a reproducible manner:  it saves the search results in a TSV file and plots them on an interactive map of the Roman Empire without any prior knowledge of programming in a matter of minutes.* 

This programme allows to extracts the output of a search query from the [Epigraphik-Datenbank  Clauss / Slaby (EDCS)](http://www.manfredclauss.de/) in a reproducible manner and saves it as a TSV file (i.e, *tab separated value*) that can be easily opened in your favourite spreadsheet software, or as a JSON file. The search results can be also plotted to a map of the Roman Empire, along with the system of Roman Provinces, roads, and cities. More on the used datasets in the *Data Sources* section.  

## Authors 
* [Brian Ballsun-Stanton, Macquarie University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0003-4932-7912)
* [Petra Heřmánková, Aarhus University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0002-6349-0540)
* [Ray Laurence, Macquarie University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0002-8229-1053)

## Cite us in your research

>**Ballsun-Stanton B., Heřmánková P., Laurence R.** Lat Epig (v2.0). GitHub. URL: https://github.com/mqAncientHistory/Lat-Epig/ DOI: 10.5281/zenodo.5211341

**License**: [GNU General Public License v3.0](https://github.com/mqAncientHistory/Lat-Epig/blob/main/LICENSE)
    
If you're using this tool in your research, <!-- Place this tag where you want the button to render. -->
<a class="github-button" href="https://github.com/mqAncientHistory/Lat-Epig" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-show-count="true" aria-label="Star mqAncientHistory/Lat-Epig on GitHub">Star</a> us on Github! (This way, we don't need to put tracking pixels into this notebook to get a sense of how many folks are using our tool!) 

If you find a bug or have a feature request, raise an <!-- Place this tag where you want the button to render. -->
<a class="github-button" href="https://github.com/mqAncientHistory/Lat-Epig/issues" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-issue-opened" data-show-count="true" aria-label="Issue mqAncientHistory/Lat-Epig on GitHub">Issue</a>!

---

# How to search for inscriptions

**!!! First of all, go to the *`Kernel menu`* from the top bar of this Notebook and choose `Restart & Run All Cells` for the search interface to load properly.**


### Text search
For simple **Text search** enter your search string into the *Text 1* field. Use Regular Expressions for more complex text string searches, or you can combine search of two terms using *Text 1* and *Text 2* fields, together with *AND*, *OR*, *NOT* operators.

*Examples:* 
* To find all inscriptions containing the term **"tumulus"**, type tumulus in the *Text 1* field.
* To search for word containing **"via"**, type "via". Warning - you will get over 12,000 outputs so it might be a good idea to use Regular Expressions to limit your search, see the following examples.
* To search for any word **starting with "via"**, type " via" (note the space). You will get terms like *via, viator, viatore* ...
* To search for any word **ending with "via"**, type "via " (note the space). You will get terms like *Fulvia, Flavia, Salvia* ...
* To search **only for word "via"**, type " via " (note the spaces).
* To find **two terms**, you can use *Text 1* + *operator* + *Text 2* fields: i.e. to find phrase "viae OR adluvione maris" type into *Text 1* "via", to *Text 2* "adl" and select the "or" *operator*. 

*More about Regular Expressions and how to use them, see the [Carpentries lesson](https://librarycarpentry.org/lc-data-intro/01-regular-expressions/)*

### Additional parameters
If you are interested in other than *Text search* or want to narrow down your searches further, you can use additional fields to specify your search. Those include:

* **EDCS-ID** 
* **Publication**
* **Place**
* **Dating from** (enter as numeric in the following format YYYY, negative values for years BC, positive values for years AD, e.g. number -300 represents year 300 BC, number 56 represents the 56 AD)
* **Dating to** (same format as Dating from)

You can further limit your search by selecting one or more of the **Provinces**, and **Inscription Genus** (custom categorisation of inscriptions applied by EDCS).

Once you are happy with your selection, press the **Get inscriptions!** button and wait for the result to show in the window below.

### Download the results

When the search is done, a total number of inscriptions found will show in the window below, together with two links to download the data in a **TSV file** and a **JSON file** format. Click on the format of your choice to download the link to your local computer. You can easily open a TSV file with your favourite spreadsheet software. For working with the JSON file we recommend using either R or Python.

Note that the *file name* in both formats contains the date of your search, source of the data and how it was accessed (EDCS via *LatEpig*) and any search parametres or their combinations you have used (*Term 1, Term 2, Dating from*...) and the number of inscriptions found. This way you will always remember what you have searched for and when you share the file with a colleague or students, they can always replicate your search to see if any new inscriptions were added to EDCS.

---

## Data sources

### Inscriptions
The Epigraphik-Datenbank Clauss / Slaby (EDCS) is a digital collection of more than 500,000 Latin inscriptions, created by Prof. Manfred Clauss, Anne Kolb, Wolfgang A. Slaby, Barbara Woitas, and hosted by the Universitat Zurich and Katolische Universitat Eichstat-Ingoldstadt. For more information see [www.manfredclauss.de](www.manfredclauss.de).

### Spatial data (used in Maps)
_1. Roman Empire Boundaries & Provinces_

[Ancient World Mapping Centre](http://awmc.unc.edu/) datasets available through (in 2024): [Cultural data](https://github.com/AWMC/geodata/tree/master/Cultural-Data). The GeoJson files are offered under the ODC Open Database License (http://opendatacommons.org/licenses/odbl/1.0/). Data is derived from the Barrington Atlas of the Greek and Roman World, and uses AWMC modifications to OpenStreetMap (https://www.openstreetmap.org/), which is under the ODC Open Database License. Previous versions were available through the AWMC website, under the Creative Commons Attribution-NonCommercial 4.0 International License.

1. Roman Empire 60 BC (provinces or extent)
1. Roman Empire in AD 14 (provinces or extent)
1. Roman Empire in AD 69 (provinces or extent)
1. Roman Empire in AD 117 (DEFAULT, provinces or extent)
1. Roman Empire in AD 200 (provinces or extent)

_2. Roman Roads_
* McCormick, M. et al. 2013. "Roman Road Network (version 2008)," DARMC Scholarly Data Series, Data Contribution Series #2013-5. DARMC, Center for Geographic Analysis, Harvard University, Cambridge MA 02138.

* [Ancient World Mapping Centre, road shapefiles](https://github.com/AWMC/geodata/tree/master/Cultural-Data/roads), The GeoJson files are offered under the ODC Open Database License (http://opendatacommons.org/licenses/odbl/1.0/). Data is derived from the Barrington Atlas of the Greek and Roman World, and uses AWMC modifications to OpenStreetMap (https://www.openstreetmap.org/), which is under the ODC Open Database License. 

_3. Cities_

The shapefile of the cities used in the map is based on Hanson, J. W. (2016a). Cities Database (OXREP databases). Version 1.0: http://oxrep.classics.ox.ac.uk/databases/cities/. [DOI: https://doi.org/10.5287/bodleian:eqapevAn8](DOI: https://doi.org/10.5287/bodleian:eqapevAn8). More info available through Hanson, J. W. (2016b). An Urban Geography of the Roman World, 100 B.C. to A.D. 300. Oxford: Archaeopress.


### Metadata for the files produced by *LatEpig*

Each TSV and JSON file contains the information from EDCS separated into 22 attributes. The [*LatEpig* Metadata description](https://github.com/mqAncientHistory/Lat-Epig/blob/main/LatEpig_Metadata_Description.txt) document in the GitHub repo describes the contents of individual attributes along with their description and their original source. 

Note that the *file name* in both formats contains the date of your search, the source of the data and how it was accessed (EDCS via *LatEpig*) and any search parameters or their combinations you have used (*Term 1, Term 2, Dating from*...) and the number of inscriptions found. This way you will always remember what you have searched for and when you share the file with a colleague or students, they can always replicate your search to see if any new inscriptions were added to EDCS. The same applies to the publication quality maps (experimental feature) produced by *LatEpig*: they all contain the search parameters by default, alongside the origin of information and credit - all in the spirit of the best research practice and FAIR data principles.


### Generation of new attributes

#### Customised cleaning of the epigraphic text and unit testing

The text of the inscription is available in three different formats as three separate attributes: 
1. ‘*inscription*’ - the original text as presented by EDCS with all original markup and symbols, including the Leiden Conventions  markup for editions of inscriptions; 
2. ‘*inscription_conservative_cleaning*’ - the result of the custom cleaning function embedded in the Lat/Epig software, producing a conservative version of the text of an inscription. The text is as close to the preserved state of the text, without restorations and expansions also known as the diplomatic edition (only the characters as they appear on the support, with minimal or no editorial intervention or interpretation)
3. ‘*inscription_interpretive_cleaning*’ - the result of the custom cleaning function embedded in the Lat/Epig software, producing an interpretative version of the text of an inscription. The text contains all restorations and expansions to obtain as rich a version of the text as possible, interpunction between sentences is not preserved. This text version is most suitable for quantitative text analysis methods and NLP.

For details, see the structure of [both cleaning functions](https://github.com/mqAncientHistory/Lat-Epig/blob/main/src/lat_epig/text_parse.py) and [their unit tests](https://github.com/mqAncientHistory/Lat-Epig/blob/main/src/lat_epig/test_inscriptions.py).

#### Unit tests
- for selected attributes, and overall functionality: [dates](https://github.com/mqAncientHistory/Lat-Epig/blob/main/src/lat_epig/test_dates.py), [language](https://github.com/mqAncientHistory/Lat-Epig/blob/main/src/lat_epig/test_language.py), [data access](https://github.com/mqAncientHistory/Lat-Epig/blob/main/src/lat_epig/test_scrape.py).

---

We welcome any feedback via the Github <a class="github-button" href="https://github.com/mqAncientHistory/Lat-Epig/issues" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-issue-opened" data-show-count="true" aria-label="Issue mqAncientHistory/Lat-Epig on GitHub">Issue</a>!


### Happy epigraphic explorations!


In [4]:
from lat_epig.interface import displayRefresh

displayRefresh()

In [5]:
import warnings
warnings.filterwarnings('ignore')
from lat_epig.interface import makeScrapeInterface, Parseargs                    # loads the user friendly GUI
makeScrapeInterface()

Text(value='', description='Text 1:')

ToggleButtons(description='Operator', options=('and', 'or', 'not'), value='and')

Text(value='', description='Text 2:')

Text(value='', description='EDCS-ID:')

Text(value='', description='Publication:')

...

Button(description='Inscription Genus...', style=ButtonStyle())

SelectMultiple(description='Inscription genus / personal status:', layout=Layout(display='none'), options=(Non…

SelectMultiple(description='and not:', layout=Layout(display='none'), options=(None, 'Augusti/Augustae', 'carm…

Button(description='Get inscriptions!', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

# How to display an interactive map

*Once you have found your inscriptions, you can display them on here on an interactive map of the Roman Empire. You can then interact with the results and click on the inscriptions to check their text and other available metadata.*

To display an interactive map, there are three simple steps: 
1. Click on the **Update Data File List** to get the most recent searches.
2. Select the desired search output from the **Data File** dropdown.
3. Click on the **Refresh Interactive Map Output** to create a new interactive map and wait a few seconds. The new interactive map should show in the window below. Depending on the number of search outputs, this may take a minute or few (if you are trying to display more than 1000+ inscriptions).

You can zoom-in or zoom-out to cluster or decluster inscriptions and click on the individual markers to display the available data directly from the EDCS database.

***Note:* the interactive map is only for viewing!** If you wish to publish your map, you can use the experimental **[Publish LatEpig Map](Map.ipynb)** feature that allows for adding and changing the individual layers and map layout according to your needs. Moreover, it creates a reproducible research output with that can be easily replicated in the future as all the search parameters are printed to the map itself.


In [6]:
from lat_epig.interactive_map_interface import make_i_map_interface
make_i_map_interface()

Button(description='Update Data File List', layout=Layout(width='max-content'), style=ButtonStyle())

Dropdown(description='Data File', layout=Layout(width='max-content'), options=(('2023-04-24-term1_petra-248.ts…

Button(description='Refresh Interactive Map!', layout=Layout(width='max-content'), style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…